# Genetic Algorithms for Logistic Regression Feature Selection

<hr style="border:5px solid green"> </hr>

### Background

Microarray data is data generated from measuring the "expression of a gene (s)". Generally, the idea is to predict an attribute of the cell sample (cancerous cell, antibiotic resistance, etc), and see which levels of what gene expression are correlated or can be used to predict the attribute.

It is easy to measure many gene expressions at once, but sometimes very hard to get many different tissue/cell samples. This results in a very large feature set and a much smaller observation count. ( column count >>> row count )

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

For this example, we will use a UCI public microarray dataset here for a type of cancerous cell prediction.

https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq


Please navigate to `Data Folder` and download the zip/tar file. When that is downloaded, unzip the file and it should contain two files `data.csv` and `labels.csv`.


In [ ]:
data_dir = os.path.join('TCGA-PANCAN-HiSeq-801x20531')
data_file = os.path.join(data_dir, 'data.csv')
label_file = os.path.join(data_dir, 'labels.csv')

ma_df = pd.read_csv(data_file)
targets = pd.read_csv(label_file)

<hr style="border:5px solid red"> </hr>

> Note: this is a CONTRIVED example. We use it for illustration on using a genetic algorithm for feature selection. We can imagine using this for hyperparameter selection or even model selection as needed.  However, know that with each individual we test, we are performing a statistical hypothesis test.

> It is recommended to use NON target dependent methods for feature selection first, like PCA. Or use limited iteration methods like forward/backwards selection at least.  This example will scour nearly every possible feature combination for one that performs well.

> At the very least, we can incorporate lots of regularization and penalize it.

<hr style="border:5px solid red"> </hr>

In [ ]:
# Too much for logistic regression.
# We filter for illustrative purposes.
ma_df = ma_df.iloc[:, 0:1001]

# Add target column back into dataset
# We also only want to predict the 'BRCA' type of cancer to keep this to a single class prediction problem:
#.   our target will be: 'BRCA' and 'not BRCA'
ma_df['target'] = targets['Class'] == 'BRCA'
ma_df['target'] = ma_df['target'].astype(int)

In [ ]:
ma_df['target'].value_counts()

In [ ]:
ma_df.head()

In [ ]:
ma_df['target'].value_counts()

### Setup Validation, Test, and Train set.

In [ ]:
train_and_test, valid = train_test_split(ma_df, test_size=0.10, random_state=42)

In [ ]:
test, train  = train_test_split(ma_df, test_size=0.10, random_state=42)

In [ ]:
print('valid_df shape: {}'.format(valid.shape))
print('test_df shape: {}'.format(train.shape))
print('train_df shape: {}'.format(test.shape))

### Setup some parameters

In [ ]:
target_col = 'target'
feature_cols = [x for x in ma_df.columns if 'gene_' in x]
print('Number of genes (features): {:,}'.format(len(feature_cols)))

# GA Params
pop_size = 100
individual_length = len(feature_cols)
p_mutation = 50/n_genes
selection_strength = 0.10
generations = 25

## Setup Population Functions

<hr style="border:3px solid green"> </hr>

1. Population Initialization

2. Fitness Evaluation

3. Selection

4. Recombination

5. Mutation


## Population Initialization

In [ ]:
# Population initialization:

def initialize_pop(pop_size, individual_length):
    """
    Description: Return a randomly initialized population.
    Args:
     -pop_size
     -individual_length
    Return: numpy array. Array of N-popuation vectors, each of length 'individual_length', N=pop_size.
    """
    population = np.random.choice([1, 0], size=(pop_size, individual_length))

    return population

In [ ]:
population = initialize_pop(pop_size, individual_length)

In [ ]:
print(population.shape)

In [ ]:
population[0:3, 0:50]

## Fitness Evaluation

In [ ]:
# Fitness Evaluation

def get_individual_fitness(individual, df, features):
    """
    Description: Evaluate an individual solution fitness.
    Args:
     - individual: np.array, size (1 x individual_length).
     - x_data: np.array.
     - features: list. list of feature names.
    Return: fitness.
    """
    # Define classifier
    clf = linear_model.LogisticRegression(penalty='l1', solver='liblinear', tol=1e-6, max_iter=int(1e6), C=0.01)
    
    # Get features from individual
    gene_cols = [x for x, indicator in zip(features, individual) if indicator==1]
    
    # Fit on x_dataset
    clf.fit(df.loc[:, gene_cols], df[target_col])
    
    # Get f1-score
    pred_outcomes = clf.predict(df.loc[:, gene_cols])
    f1_fitness = f1_score(df[target_col].values, pred_outcomes)
    
    return f1_fitness

## Selection Function

In [ ]:
# Selection Function

def selection(population, pop_fitness, selection_strength):
    """
    Description: Perform Selection on Population- return a subpopulation of the highest fit population.
    Args:
     - population: np.array, size (pop_size x individual_length).
     - pop_fitness: np.array. Array of fitness values corresponding to each individual in prior population.
     - selection_strength. Float. 0<x<1, % of top individuals to keep. Lower is more restrictive.
    Return: np.array. size ( floor(selection_strength * population) x individual_length )
    """
    # Figure out how many to save:
    num_to_save = int(np.floor(selection_strength * population.shape[0]))
    
    # Get indices of N top fit individuals (smallest MSE is best)
    # individuals_to_save = pop_fitness.argsort()[:num_to_save]
    
    # For maximizing fitness:
    individuals_to_save = pop_fitness.argsort()[-num_to_save:][::-1]
    
    # Save top fit parent population
    parents = population[individuals_to_save, :]
    
    return parents

## Recombination Function

In [ ]:
# Recombination Function

def recombination(parent_pop, pop_size):
    """
    Description: Recombine parents to create new children solutions.
    Args:
     - parent_pop: np.array, size (parent_size x individual_length).
     - pop_size: Integer. Total population to make up
    Return: np.array. size ( pop_size x individual_length )
    """
    # Figure out how many children to create
    num_children = pop_size - parent_pop.shape[0]
    
    # Initialize Children
    children = np.zeros(shape=(num_children, parent_pop.shape[1]))
    
    # Loop and create each child
    for c in range(num_children):
        # Randomly get two parents
        parents = parent_pop[np.random.randint(0,parent_pop.shape[0],2)]
        
        # Select a random crossover location
        crossover_pt = np.random.randint(1, high=(parent_pop.shape[1] - 1))
        
        # Create child
        children[c, :crossover_pt] = parents[0, :crossover_pt]
        children[c, crossover_pt:] = parents[1, crossover_pt:]
        
    return children

## Mutation Function

In [ ]:
# Mutation Function

def mutation(individual, p_mutation):
    """
    Description: Mutate Child Population.
    Args:
     - individual: np.array, size (1 x individual_length).
     - p_mutation: Float. 0<x<1. Probability of mutation of each sub-parameter of a single individual.
    Return: np.array. size ( 1 x individual_length )
    """
    # Generate a probability vector, same size as individual using the uniform distribution.
    prob_vector = np.random.uniform(low=0.0, high=1.0, size=individual_length)
    
    # Find where to mutate individual
    ix_to_mutate = np.argwhere(prob_vector < p_mutation)
    
    # Mutate individual
    if ix_to_mutate.size > 0:
        new_vals = np.random.choice([0, 1], size=ix_to_mutate.shape)
        individual[ix_to_mutate] = new_vals
        
    return individual

## Start Genetic Algorithm

In [ ]:
best_fitness_sequence = []
best_fitness = 0.0

for g in range(generations):
    print('Starting generation {:,} out of {:,}: {}'.format(g + 1, generations, best_fitness))
    
    # Get fitness of population
    pop_fitness = np.apply_along_axis(get_individual_fitness, 1, population, ma_df, feature_cols)
    
    # Save best fitness
    best_fitness = np.max(pop_fitness)
    best_ix = np.argmax(pop_fitness)
    best_fitness_sequence.append(best_fitness)
    
    # Get parents
    parents = selection(population, pop_fitness, selection_strength)
    
    # Create children from parents
    children = recombination(parents, pop_size)
    
    # Mutate children
    children = np.apply_along_axis(mutation, 1, children, p_mutation)
    
    # Combine parents and children
    population = np.concatenate((parents, children), axis=0)

In [ ]:
print('Best F1 score: {}'.format(best_fitness))

### Plot F1 score vs Generation

In [ ]:
g_seq = np.arange(len(best_fitness_sequence))

plt.plot(g_seq, np.array(best_fitness_sequence))
plt.xlabel('Num of Generations')
plt.ylabel('F1 Score (Fitness)')
plt.title('F1 vs Generations')